# Using the AWRA MS to run a Simulation of the AWRA-L model
There are two ways to run a simulation of the AWRA-L model in the AWRA MS
* Using the On-Demand Simulator
* Using the Server Simulator


## Server Simulation


The Server Simulation is set up very similarly to the On Demand Simulator, except it needs to write the outputs to disk as the model run progresses, as it does not hold the results in memory. This allows the Server Simulator to be the more efficient choice for longer runs over a larger extent. The same output and input maps can be used for either simulator.

This notebook goes through the following steps:

1. Import required libraries
2. Modify the model configuration <br>
 2.1 Read in the default configuration<br>
 2.2 Change forcing data<br>
 2.3 Create model output map and add nodes for saving some model outputs to netcdf files<br>
 2.4 Specify period and extents<br>
3. Put model run specification together<br>
 3.1 Instantiate the simulator<br>
 3.2 Run the model<br>
4. Visualise outputs<br>
 4.1 Import libraries<br>
 4.2 Load model outputs<br>
 4.3 Look at time slice<br>
5. Further simulation functionality<br>
 5.1 Change initial states<br>

### 1. Import required libraries

In [ ]:
from awrams.simulation import server
from awrams.utils import config_manager
from awrams.simulation.support import build_output_mapping
from awrams.utils.nodegraph import nodes
from awrams.utils.io.data_mapping import SplitFileManager

from awrams.utils import extents
from awrams.utils import datetools

In [ ]:
model_profile = config_manager.get_model_profile('awral','v6_default')


### 2. Modify the model configuration

#### 2.1 Read in  default configuration

In [ ]:
input_map = model_profile.get_input_mapping()

In [ ]:
input_map

#### 2.3 Create model output map and add nodes for saving some model outputs to netcdf files

In [ ]:
model_settings = model_profile.get_settings()

In [ ]:
# These are the default model outpus
# OUTPUTS_AVG : Weighted average of HRU outputs
# OUTPUTS_CELL : Cell level outputs (ie processes happening outside of HRU; groundwater store etc)
# OUTPUTS_HRU : HRU level outputs (ie separate output per HRU)

model_settings.OUTPUTS

In [ ]:
model = model_profile.get_model(model_settings)

In [ ]:
outpath = './results_simserver/'

# By default, build_output_mapping will generate files for all outputs - this a lot of data, so usually you will want
# to specify which variables to save
# Outputs from OUTPUTS_AVG and OUTPUTS_CELL use the same names as in model_settings.OUTPUTS, while OUTPUTS_HRU
# has a separate name for each HRU.
# Here we save ss and sd as weighted averages, but s0 separately from both HRUs

save_vars = ['qtot','ss','sd','s0_hrusr','s0_hrudr']

# If you want to reinitialise a run from existing states, they will need to be saved at 64bit resolution.
# We will use the 'save_states_freq' argument to create 'snapshots' of states on a monthly basis so you don't have
# to write too much data to disk...

omap = build_output_mapping(model, outpath, save_vars = save_vars, save_states_freq = 'M')

# The updated output map contains write_to_annual_ncfile nodes for the variables we specified above, 
# as well as write_to_ncfile_snapshot for the states
# It's possible to further manipulate this map directly, but in most cases you won't need to

omap

#### 2.4	Define the required period and spatial extent
Because it is the Server Simulator the extent can be defined as the full continent which is the default, 
but for a longer period that one day

In [ ]:
period = datetools.dates('dec 2010 - jan 2011')
edef = extents.get_default_extent()

### 3. Put the model run specification together 

#### 3.1 Instantiate the server simulator

In [ ]:
sim = server.SimulationServer(model)

#### 3.2 Run the simulation

In [ ]:
# Run with 'clobber' == True; this will delete all data from previous runs in this folder.
# This value is False by default (to avoid destroying data), but we'll turn it on here just in case you've
# run the training notebooks previously...

sim.run(input_map,omap,period,edef,clobber=True)

### 4. Visualise outputs

Because ouputs are written out, we can use the process described in the [Visualisation] notebook

[Visualisation]: ../Visualisation/Visualisation.ipynb

#### 4.1 Import visualisation libraries

In [ ]:
import awrams.visualisation.vis as vis
import awrams.visualisation.results as res

import awrams.utils.extents as extents


#### 4.2 Load model outputs

In [ ]:
results = res.load_results('./results_simserver/')

In [ ]:
results.variables

#### 4.3 Look at a time slice

In [ ]:
%matplotlib inline

In [ ]:
results[:,'1 jan 2011',:].spatial()

### 5. Further simulation functionality

These will apply equally to the OnDemand and Server options

#### 5.1 How to change the initial states


In [ ]:
# For this example we will use the states output from the previous run

initial_states_path = './results_simserver/states/'

In [ ]:
# Update our input mapping to use the existing states
# init_state_from_ncfile takes an optional date argument; by default it will read the states for the day prior to 
# the start of the run period, which is equivalent to resuming the run from where it left off..

for k in model.get_state_keys():
    input_map['init_' + k] = nodes.init_state_from_ncfile(initial_states_path,k+'*.nc',k)

In [ ]:
# Inspect the updated nodes

dict([(k,v) for k,v in input_map.items() if k.startswith('init_')])

In [ ]:
# Build a new output map pointing to a different path.  This time we won't bother saving out the states

outpath = './results_simserver_initstates/'

save_vars = ['qtot','s0_hrusr','s0_hrudr']

omap = build_output_mapping(model, outpath, save_vars = save_vars)

In [ ]:
# Change the period so that this sim starts halfway through the previous run; this way we can verify we generate the 
# same outputs as if we'd done the full run

period = datetools.dates('jan 2011 - mar 2011')

In [ ]:
sim = server.SimulationServer(model)

sim.run(input_map,omap,period,edef,clobber=True)

### Settings states from in-memory data

In [ ]:
# States don't need to be loaded from disk; you can generate them in memory, allowing for custom states etc

# In this example we'll read the arrays from the previous states files, but pass in the data in-memory
# Obviously this isn't particularly useful by itself, but allows for modification of the data before
# it reaches the new simulation...

# Specify the day prior to the start of our run period 
# This is the same date that would be read by default in init_state_from_ncfile
state_date = period[0] - 1

data_map = {}

for k in model.get_state_keys():    
    # Open each of our previous states
    sfm = SplitFileManager.open_existing(initial_states_path,k+'*.nc',k)
    
    # Read the data for our specified day, for the full extent
    data_map['init_' + k] = sfm.get_data(state_date,edef)
    
# nodes.init_states_from_dict is a convenience function that updates an input map in place with initial state data
# It also takes an extent object as an argument so that it knows which spatial area the data refers to

nodes.init_states_from_dict(input_map,data_map,edef)

In [ ]:
# Have a quick look at data for a single cell (200,200)

[(k,v.args['data'][200,200]) for k,v in input_map.items() if k.startswith('init_')]

In [ ]:
sim = server.SimulationServer(model)

outpath = './results_simserver_initstates_dict/'
omap = build_output_mapping(model, outpath, save_vars = save_vars)

sim.run(input_map,omap,period,edef)

## End notebook